In [1]:
#14.7.2021

from sklearn import svm
from sklearn.metrics import accuracy_score
import numpy as np
import cv2
import os
from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans
import cv2 
import matplotlib.pyplot as plt
%matplotlib inline

# Import datasets, classifiers and performance metrics
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split


In [19]:
train_path = 'iriseye1/train'  # Folder Names are Parasitized and Uninfected
training_names = os.listdir(train_path)

# Get path to all images and save them in a list
# image_paths and the corresponding label in image_paths
image_paths = []
image_classes = []
class_id = 0

#To make it easy to list all file names in a directory let us define a function
#
def imglist(path):
	return [os.path.join(path, f) for f in os.listdir(path)]

#Fill the placeholder empty lists with image path, classes, and add class ID number
for training_name in training_names:
	dir = os.path.join(train_path, training_name)
	class_path = imglist(dir)
	image_paths+=class_path
	image_classes+=[class_id]*len(class_path)
	class_id+=1

# Create feature extraction and keypoint detector objects
    #SIFT is not available anymore in openCV    
# Create List where all the descriptors will be stored
des_list = []

In [20]:
sift = cv2.xfeatures2d.SIFT_create()

for image_path in image_paths:
	im = cv2.imread(image_path)
	img = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
	# Adaptive Gaussian Thresholding
	#th1 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
	# Otsu's thresholding
	#ret2,th2 = cv2.threshold(th1,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
	# Otsu's thresholding after Gaussian filtering
	#blur = cv2.GaussianBlur(th2,(5,5),0)
	#ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
	#img = img.astype('uint8')
	kpts, des = sift.detectAndCompute(img, None)
	des_list.append((image_path, des))
print(des_list)
print("---------------------")
print(des)

[('iriseye1/train/2/002L_2.jpg', array([[24.,  3.,  0., ...,  0.,  0.,  0.],
       [47.,  1.,  0., ...,  1.,  0.,  0.],
       [ 1.,  1.,  0., ...,  1.,  0.,  0.],
       ...,
       [ 1.,  0.,  0., ...,  0.,  0.,  1.],
       [40.,  0.,  0., ...,  2.,  1.,  0.],
       [42.,  1.,  0., ...,  4.,  2.,  0.]], dtype=float32)), ('iriseye1/train/2/002L_1.jpg', array([[ 1.,  0.,  0., ..., 22.,  1.,  1.],
       [26.,  2.,  0., ...,  0.,  0.,  0.],
       [ 1.,  1.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 3.,  1.,  0., ...,  3.,  0.,  2.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [36.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)), ('iriseye1/train/2/002L_3.jpg', array([[ 1.,  0.,  0., ..., 20.,  0.,  0.],
       [ 8.,  4.,  0., ...,  0.,  0.,  0.],
       [ 1., 16., 51., ...,  0.,  0.,  0.],
       ...,
       [55.,  3.,  0., ...,  1.,  0.,  1.],
       [ 3.,  1.,  0., ...,  1.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  1.]], dtype=float32)), ('iriseye1/train/3/00

In [21]:
des[0]

array([  1.,   1.,   0.,   0.,   0.,   0.,   0.,   0., 178.,   5.,   0.,
         0.,   0.,   0.,   0.,   3., 180.,  14.,   0.,   1.,  12.,   0.,
         0.,   7.,  10.,   1.,   0.,   4.,  42.,   1.,   0.,   1.,   1.,
         0.,   0.,   0.,   0.,   0.,   0.,   0., 180.,  15.,   0.,   0.,
         0.,   0.,   0.,   1., 180.,  44.,   0.,   2.,   8.,   2.,   0.,
         5.,  16.,   3.,   0.,   3.,  25.,  12.,   4.,   3.,   1.,   0.,
         0.,   0.,   0.,   0.,   0.,   0., 180.,   2.,   0.,   0.,   0.,
         0.,   0.,   9., 180.,   5.,   0.,   0.,   4.,   5.,   1.,  36.,
        19.,   0.,   0.,   0.,  16.,  22.,   3.,   6.,   1.,   0.,   0.,
         0.,   0.,   0.,   0.,   0., 162.,   5.,   0.,   0.,   0.,   0.,
         0.,   1., 180.,  15.,   0.,   0.,   6.,   1.,   0.,   5.,  18.,
         1.,   0.,   0.,  24.,   6.,   0.,   1.], dtype=float32)

In [22]:
des.shape   #that are keypoints, you have to clustering them----> results will be clusters of keypoints---> each cluster is feature

(173, 128)

In [23]:
from sklearn.cluster import KMeans
kmeans = KMeans(init="k-means++", n_clusters=100, n_init=4,random_state=0)
#X=descriptors
kmeans.fit(des)
visual_words = kmeans.cluster_centers_
print(kmeans)
print(visual_words)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=100, n_init=4, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)
[[ 1.0000000e+00  1.0000000e+01  1.9000000e+01 ...  0.0000000e+00
   0.0000000e+00  1.2000000e+01]
 [ 6.1900002e+01  1.2000008e+00  0.0000000e+00 ...  9.5000000e+00
   1.1000001e+00  0.0000000e+00]
 [ 1.6000000e+01  3.1000000e+01  9.5000000e+00 ...  6.5000000e+00
   1.0000000e+00  5.0000000e+00]
 ...
 [ 6.9000000e+01  5.0000000e+00  0.0000000e+00 ...  4.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 3.5000000e+01  0.0000000e+00  0.0000000e+00 ...  2.6000000e+01
   0.0000000e+00  4.0000000e+00]
 [ 1.2857151e+00  4.2857265e-01 -4.7683716e-07 ...  4.2857170e-01
   2.8571463e-01  4.4285712e+00]]


In [24]:
visual_words.shape   #each cluster is feature---> this=visual_words is test mn.

(100, 128)

In [25]:
visual_words[0]  #first feature, for first image=first cluster

array([  1.,  10.,  19.,   5.,   2.,   1.,  11.,  15.,  10.,  33.,  20.,
         2.,   0.,   4.,  50.,  77.,   5.,   7.,   5.,   8.,  12.,  40.,
        61.,  15.,  17.,   1.,   0.,   0.,   1.,   6.,  23.,  55.,  43.,
         0.,   0.,   0.,   1.,   1.,  24.,  78., 131.,   8.,   1.,   1.,
         7.,  41., 119., 131.,  11.,   3.,   1.,   2.,  60., 131., 131.,
        42.,   1.,   0.,   0.,   0.,   2.,  25.,  72., 131.,  40.,   0.,
         0.,   0.,   0.,   1.,  79., 111., 131.,  80.,  20.,  16.,  12.,
         9.,  26.,  81.,  35.,  31.,  27.,  82., 111.,  72.,  18.,  21.,
        32.,   2.,   0.,   2.,   6.,  12.,  26., 125.,  14.,   7.,   1.,
         0.,   1.,   2.,  44.,  78.,  20.,  17.,  11.,  16.,  12.,   6.,
        34.,  33.,  26.,  19.,  16.,  46.,  22.,   1.,   1.,   2.,  57.,
        55.,  15.,   1.,   0.,   0.,   0.,  12.], dtype=float32)

In [11]:
#if (descriptor is not None):https://towardsdatascience.com/bag-of-visual-words-in-a-nutshell-9ceea97ce0fb
histogram = build_histogram(visual_words, kmeans)
preprocessed_image.append(histogram)

NameError: name 'build_histogram' is not defined

In [26]:
from scipy.cluster.vq import kmeans, vq

k = 100  #k means with 100,600,500,300 clusters gives lower accuracy 
voc, variance = kmeans(des, k, 1) 

# Calculate the histogram of features and represent them as vector
#vq Assigns codes from a code book to observations.
im_features = np.zeros((len(image_paths), k), "float32")
for i in range(len(image_paths)):
	words, distance = vq(des_list[i][1],voc)
	for w in words:
		im_features[i][w] += 1

In [27]:
from sklearn.preprocessing import StandardScaler
im_features = StandardScaler().fit_transform(im_features)

In [28]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(200, ), random_state=1, max_iter=100000, activation='logistic')       #, batch_size=400)


In [29]:
# evaluate model using 10-fold cross validation
from sklearn.model_selection import StratifiedKFold, cross_val_score
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)  #crossvalidation for better results, n_splits=10 cannot be greater than the number of members in each class
results = cross_val_score(clf, im_features, np.array(image_classes), cv=kfold)
clf.fit(im_features, np.array(image_classes))

MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(200,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=100000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [30]:
#Joblib dumps Python object into one file
from sklearn.externals import joblib
joblib.dump((clf, training_names, k, voc), "iriseye1cnn.pkl", compress=3) #izzvzela stdSlr

['iriseye1cnn.pkl']